# 動物標本を用いた局所冷却実験の解析

## 解析するデータ

- 2005-04-16-1/S0025
- 2005-08-24-2/S0011
- 2006-02-18-1/S0011

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import copy
from scipy import ndimage
%matplotlib inline

In [2]:
from opmap.RawCam import RawCam
from opmap.VmemMap import VmemMap
from opmap.PhaseMap import PhaseMap
from opmap.PhaseVarianceMap import PhaseVarianceMap
from opmap.APDMap import APDMap

## windowサイズの決定

In [7]:
window_sizes = np.arange(3, 26)

In [14]:
cam = RawCam(path='/mnt/recordings/ExperimentData/2005-04-16-1/C001S0025@',
                 cam_type='sa4',
                 image_width=256, image_height=256,
                 frame_start=0, frame_end=100)

In [15]:
for window_size in window_sizes:
    result_name = '/mnt/recordings/AnalysisResult/Seno/opmap/HRS2018/20171006/filter/2005-04-16-1/'
    if not os.path.exists(result_name):
        os.makedirs(result_name)
    plt.imsave(result_name+'window_%s.png' % window_size, ndimage.gaussian_filter(cam.data[50, :, :], sigma=window_size))

## windowサイズ 20で解析

In [6]:
directories = ['2005-04-16-1/C001S0025@', '2005-08-24-2/C001S0011@', '2006-02-18-1/C001S0011@']
threshold = 0.4

In [ ]:
for directory in directories:
    cam = RawCam(path='/mnt/recordings/ExperimentData/%s' % directory,
                 cam_type='sa4',
                 image_width=256, image_height=256,
                 frame_start=0, frame_end=-1)
    cam.setIntROI(val_min=100)

    result_parent = '/mnt/recordings/AnalysisResult/Seno/opmap/HRS2018/20171006/%s' % directory
    if not os.path.exists(result_parent):
        os.makedirs(result_parent)

    vmem = VmemMap(cam)
    vmem.smooth(size=20)
    vmem.saveImage(os.path.join(result_parent, 'vmem'))
    np.save(os.path.join(result_parent, 'vmem.npy'), vmem.data)
    cmd = 'ffmpeg -r 30 -y -i "{0}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "{0}/movie.mp4"'.format(os.path.join(result_parent, 'vmem'))
    subprocess.call(cmd, shell=True)

    
    pmap = PhaseMap(vmem, shrink=2)
    pmap.saveImage(os.path.join(result_parent, 'pmap'))
    cmd = 'ffmpeg -r 30 -y -i "{0}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "{0}/movie.mp4"'.format(os.path.join(result_parent, 'pmap'))
    subprocess.call(cmd, shell=True)

    pvmap = PhaseVarianceMap(pmap, size=3)
    pvmap.saveImage(os.path.join(result_parent, 'pvmap'))
    np.save(os.path.join(result_parent, 'pvmap.npy'), pvmap.data)
    cmd = 'ffmpeg -r 30 -y -i "{0}/%06d.png" -c:v libx264 -pix_fmt yuv420p -qscale 0 "{0}/movie.mp4"'.format(os.path.join(result_parent, 'pvmap'))
    subprocess.call(cmd, shell=True)
    
    APV_array = np.zeros((128, 128))
    phase_variance_array = copy.deepcopy(pvmap.data)
    ROI = copy.deepcopy(pvmap.roi)
    core = (phase_variance_array > threshold) * 1
    core_count = np.sum(core, axis=0)
    
    for i in range(128):
        for j in range(128):
            if ROI[i,j] != 0:
                data = copy.deepcopy(phase_variance_array[:, i, j])
                time = data.shape[0]
                while np.max(data) > threshold:
                    x_pos = np.argmax(data)
                    x_max = x_pos + 1
                    x_min = x_pos - 1
                    while x_max < time and data[x_max - 1] > data[x_max]:
                        x_max += 1
                    while x_min > -1 and data[x_min] < data[x_min + 1]:
                        x_min -= 1
                    data[x_min+2:x_max] = 0
                APV_array[i,j] = np.sum(data)/len(np.nonzero(data)[0])
                """
                if core_count[i,j] > 0:
                    passed_list.append(APV_array[i,j])
                else:
                    nonpassed_list.append(APV_array[i,j])
                """

    plt.imshow(core_count, cmap='jet')
    plt.colorbar()
    plt.savefig(os.path.join(result_parent, 'core.png'), dpi=300, transparent=True)
    plt.close()
    
    plt.imshow(APV_array, cmap='jet')
    plt.colorbar()
    plt.savefig(os.path.join(result_parent, 'APV.png'), dpi=300, transparent=True)
    plt.close()

## カメラ画像出力

In [16]:
for directory in directories:
    cam = RawCam(path='/mnt/recordings/ExperimentData/%s' % directory,
                 cam_type='sa4',
                 image_width=256, image_height=256,
                 frame_start=0, frame_end=100)
    cam.setIntROI(val_min=100)

    result_parent = '/mnt/recordings/AnalysisResult/Seno/opmap/HRS2018/20171006/%s' % directory
    if not os.path.exists(result_parent):
        os.makedirs(result_parent)

    cam.saveImage(os.path.join(result_parent, 'cam'))

## APDマップ作成

In [3]:
APD_directories = ['2005-04-16-1/C001S0001#', '2005-04-16-1/C001S0002#', '2005-04-16-1/C001S0003#',
                   '2005-04-16-1/C001S0004#', '2005-04-16-1/C001S0007#', '2005-04-16-1/C001S0008#']

In [8]:
for directory in APD_directories:
    print(directory)
    cam = RawCam(path='/mnt/recordings/ExperimentData/%s' % directory,
                 cam_type='sa4',
                 image_width=256, image_height=256,
                 frame_start=0, frame_end=10)
    cam.setIntROI(val_min=100)

    result_parent = '/mnt/recordings/AnalysisResult/Seno/opmap/HRS2018/20171009/%s' % directory
    if not os.path.exists(result_parent):
        os.makedirs(result_parent)

    cam.saveImage(os.path.join(result_parent, 'cam'))

    """    
    vmem = VmemMap(cam)
    vmem.smooth(size=20)
    np.save(os.path.join(result_parent, 'vmem.npy'), vmem.data)
    APD = APDMap(vmem)
    np.save(os.path.join(result_parent, 'APD.npy'), APD.data)
    """

2005-04-16-1/C001S0001#
2005-04-16-1/C001S0002#
2005-04-16-1/C001S0003#
2005-04-16-1/C001S0004#
2005-04-16-1/C001S0007#
2005-04-16-1/C001S0008#
